In [1]:
%cd /hdd/yuchen/StableDiffusionReconstruction/codes/utils

/hdd/yuchen/StableDiffusionReconstruction/codes/utils


In [2]:
import argparse
import os
import numpy as np
import pandas as pd
from nsd_access import NSDAccess
import scipy.io
import argparse, os
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from tqdm import tqdm
from einops import repeat
from torch import autocast
from contextlib import nullcontext
from pytorch_lightning import seed_everything
from nsd_access import NSDAccess
from PIL import Image
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
import scipy.io
import random

In [3]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    imgidx = (0, 10)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early 
    
    method = 'cvpr'
    
opt = Opt()

In [4]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.cuda.empty_cache()

## make_subjmri.py

In [2]:
import pandas as pd
caption = pd.read_csv('/hdd/yuchen/cs292/things_concepts.tsv', sep='\t')

In [6]:
from os.path import join as pjoin

betas_csv_dir = '/hdd/yuchen/cs292/fmri/betas_csv/'
sub = '01'
    
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)

train_idx = stimdata[stimdata['trial_type'] == 'train'].index.tolist()
test_idx = stimdata[stimdata['trial_type'] == 'test'].index.tolist()
    
train_fmri = responses[train_idx]
test_fmri = responses[test_idx]
    
train_labels = stimdata[stimdata['trial_type'] == 'train']['stimulus']
test_labels = stimdata[stimdata['trial_type'] == 'test']['stimulus']
    
roi_idx = voxdata[(voxdata['V1'] == 1) | (voxdata['V2'] == 1) | (voxdata['V3'] == 1) | (voxdata['hV4'] == 1) ]['voxel_id'].tolist()
train_fmri = train_fmri.iloc[roi_idx]
test_fmri = test_fmri.iloc[roi_idx]

test = stimdata[stimdata['trial_type'] == 'test']
test_fmri_avg = []
test_labels = test['stimulus'].unique()
for stim in test_labels:
    repeated_trial = test[test['stimulus'] == stim].index.tolist()
    test_fmri_avg.append(test_fmri[repeated_trial].mean(axis=1).to_numpy())
test_fmri_avg = np.array(test_fmri_avg).T

del responses, voxdata, stimdata

train_img = [] 
first_img_path = os.path.join('/hdd/yuchen/cs292/images/', train_labels.iloc[0])
with Image.open(first_img_path) as first_img:
    first_img_array = np.array(first_img)
    img_shape = first_img_array.shape
        
train_img = np.empty((len(train_labels), *img_shape), dtype=first_img_array.dtype)
for i, label in enumerate(train_labels):
    img_path = os.path.join('/hdd/yuchen/cs292/images/', label)
    with Image.open(img_path) as img:
        train_img[i] = np.array(img)

test_img = []  
first_img_path = os.path.join('/hdd/yuchen/cs292/images/', test_labels[0])
with Image.open(first_img_path) as first_img:
    first_img_array = np.array(first_img)
    img_shape = first_img_array.shape
        
test_img = np.empty((len(test_labels), *img_shape), dtype=first_img_array.dtype)
for i, label in enumerate(test_labels):
    img_path = os.path.join('/hdd/yuchen/cs292/images/', label)
    with Image.open(img_path) as img:
        test_img[i] = np.array(img)
train_fmri, test_fmri =  train_fmri.to_numpy(), test_fmri.to_numpy()
train_img_label_all, test_img_label_all = train_labels.tolist(), test_labels.tolist()

In [7]:
train_label_caption = []
for i in train_labels.tolist():
    temp_i = i.split('.')[0].rsplit('_')[:-1]
    temp = ''
    for str in range(len(temp_i)):
        temp += temp_i[str]
        if str != len(temp_i)-1:
            temp+= '_'
    train_label_caption.append(caption[caption.uniqueID == temp]['Definition (from WordNet, Google, or Wikipedia)'].tolist()[0])

test_label_caption = []
for i in test_labels.tolist():
    temp_i = i.split('.')[0].rsplit('_')[:-1]
    temp = ''
    for str in range(len(temp_i)):
        temp += temp_i[str]
        if str != len(temp_i)-1:
            temp+= '_'
    test_label_caption.append(caption[caption.uniqueID == temp]['Definition (from WordNet, Google, or Wikipedia)'].tolist()[0])

In [8]:
betas_tr = train_fmri
betas_ave_te = test_fmri_avg
train_label_caption = np.array(train_label_caption)
test_label_caption = np.array(test_label_caption)

In [9]:
print('train_fmri', train_fmri.shape)
print('train_img', train_img.shape)
print('train_labels', train_labels.shape)
print('train_label_caption', train_labels.shape)

print('test_fmri_avg', test_fmri_avg.shape)
print('test_img', test_img.shape)
print('test_labels', test_labels.shape)
print('test_label_caption', test_labels.shape)

train_fmri (3198, 8640)
train_img (8640, 320, 320, 3)
train_labels (8640,)
train_label_caption (8640,)
test_fmri_avg (3198, 100)
test_img (100, 320, 320, 3)
test_labels (100,)
test_label_caption (100,)


## img2feat_sd1.py

In [10]:
def load_model_from_config(config, ckpt, gpu, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda(f"cuda:{gpu}")
    model.eval()
    return model

def load_img_from_arr(img_arr,resolution):
    image = Image.fromarray(img_arr).convert("RGB")
    w, h = resolution, resolution
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [11]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    save_folder = '/hdd/yuchen/cs292/test/'
    imgidx = (0, 73000)
    gpu = 0
    seed = 42
    
opt = Opt()

In [12]:
# from PIL import Image
# import os

# # Define the working directory
# working_dir = "/hdd/yuchen/cs292/images"  # Replace with the path of your working directory

# new_size = (320, 320)

# for root, dirs, files in os.walk(working_dir):
#     for file in files:
#         if file.lower().endswith(('.png', '.jpg', '.jpeg')):
#             # Construct the full file path
#             file_path = os.path.join(root, file)
#             try:
#                 # Open the image
#                 with Image.open(file_path) as img:
#                     # Resize the image
#                     resized_img = img.resize(new_size)
#                     # Save the resized image back to its path
#                     resized_img.save(file_path)
#             except Exception as e:
#                 print(f"Error resizing image {file_path}: {e}")

# print("Image resizing completed.")

In [13]:
seed_everything(opt.seed)
imgidx = opt.imgidx
gpu = opt.gpu
resolution = 320
batch_size = 1
ddim_steps = 50
ddim_eta = 0.0
strength = 0.8
scale = 5.0
nsda = NSDAccess(opt.root_folder + 'nsd/')
config = opt.root_folder + 'StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/configs/stable-diffusion/v1-inference.yaml'
ckpt = opt.root_folder + 'StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt'
config = OmegaConf.load(f"{config}")
torch.cuda.set_device(gpu)
os.makedirs(f'{opt.save_folder}nsdfeat/init_latent/', exist_ok=True)
os.makedirs(f'{opt.save_folder}nsdfeat/c/', exist_ok=True)

Global seed set to 42


In [14]:
gpu=0

In [15]:
# Load moodels
precision = 'autocast'
precision_scope = autocast if precision == "autocast" else nullcontext
model = load_model_from_config(config, f"{ckpt}", gpu)
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
sampler = DDIMSampler(model)
sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)
assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(strength * ddim_steps)
print(f"target t_enc is {t_enc} steps")

Loading model from /hdd/yuchen/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode


/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.17.self_attn.v_proj.weight', 'vision_model.encoder.layers.3.self_attn.k_proj.weight', 'vision_model.encoder.layers.20.mlp.fc1.weight', 'vision_model.encoder.layers.21.mlp.fc2.weight', 'vision_model.encoder.layers.23.self_attn.out_proj.weight', 'vision_model.encoder.layers.2.layer_norm1.bias', 'vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_model.encoder.layers.7.layer_norm1.bias', 'vision_model.encoder.layers.17.self_attn.q_proj.weight', 'vision_model.encoder.layers.21.layer_norm1.weight', 'vision_model.encoder.layers.2.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.mlp.fc1.bias', 'vision_model.encoder.layers.6.self_attn.q_proj.bias', 'vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.12.mlp.fc1.weight', 'vision_model.encoder.layers.5.self_attn.q_proj.bias', 'vision_model.encoder.layers

target t_enc is 40 steps


In [52]:
# %%capture
# Sample
opt.save_folder = '/hdd/yuchen/cs292/train/'
for s in range(0, train_img.shape[0]):
    print(f"Now processing image {s:06}", end='\r')   
    p=np.array(train_labels)[s].split('.')[0].split('_')[0] + ', '+ \
        train_label_caption[s]
    prompt = [p]
    img_temp = train_img[s]/(255.0/2)-1.
    
    init_image = torch.from_numpy(np.expand_dims(img_temp.transpose(2,0,1),0)).to(device).to(torch.float32)
    init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
    
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                uc = model.get_learned_conditioning(batch_size * [""])
                c = model.get_learned_conditioning(prompt).mean(axis=0).unsqueeze(0)

    init_latent = init_latent.cpu().detach().numpy().flatten()
    c = c.cpu().detach().numpy().flatten()
    np.save(f'{opt.save_folder}nsdfeat/init_latent/{s:06}.npy',init_latent)
    np.save(f'{opt.save_folder}nsdfeat/c/{s:06}.npy',c)

KeyboardInterrupt: 

## make_subjstim.py

In [9]:
opt.subject

'subj02'

In [17]:
opt.featname = 'init_latent'
opt.use_stim = 'each'

subject= opt.subject
use_stim = opt.use_stim
featname = opt.featname
topdir = opt.root_folder + 'nsdfeat/'
savedir = f'{topdir}/subjfeat/'
featdir = f'{topdir}/{featname}/'

nsd_expdesign = scipy.io.loadmat(opt.root_folder + 'nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

    # Note that most of them are 1-base index!
    # This is why I subtract 1
sharedix = nsd_expdesign['sharedix'] -1 

if use_stim == 'ave':
    stims = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_ave.npy')
else: # Each
    stims = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims.npy')
    

In [18]:
feats = []
tr_idx = np.zeros(len(stims))

for idx, s in tqdm(enumerate(stims)): 
    if s in sharedix:
        tr_idx[idx] = 0
    else:
        tr_idx[idx] = 1    
    feat = np.load(f'{featdir}/{s:06}.npy')
    feats.append(feat)

feats = np.stack(feats)    

os.makedirs(savedir, exist_ok=True)

feats_tr = feats[tr_idx==1,:]
feats_te = feats[tr_idx==0,:]
np.save(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_tridx.npy',tr_idx)
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_tr.npy',feats_tr)
np.save(f'{savedir}/{subject}_{use_stim}_{featname}_te.npy',feats_te)

27750it [00:05, 4880.60it/s]


## ridge.py

In [17]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    imgidx = (0, 73000)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early   TODO: how do they select that ?
    
opt = Opt()

In [25]:
opt.target = 'init_latent'
opt.roi = 'early'

target = opt.target
roi = opt.roi

backend = set_backend("numpy", on_error="warn")
subject=opt.subject

if target == 'c' or target == 'init_latent': # CVPR
    alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]
else: # text / GAN / depth decoding (with much larger number of voxels)
    alpha = [10000, 20000, 40000]

ridge = RidgeCV(alphas=alpha)

preprocess_pipeline = make_pipeline(
        StandardScaler(with_mean=True, with_std=True),
    )
pipeline = make_pipeline(
        preprocess_pipeline,
        ridge,
)    
mridir = f'{opt.root_folder}mrifeat/{subject}'
featdir = f'{opt.root_folder}nsdfeat/subjfeat'
savedir = f'{opt.root_folder}decoded/{subject}/'
os.makedirs(savedir, exist_ok=True)

In [26]:
X = []
X_te = []
for croi in [roi]:
    if 'conv' in target: # We use averaged features for GAN due to large number of dimension of features
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32")
    else:
        cX = np.load(f'{mridir}/{subject}_{croi}_betas_tr.npy').astype("float32")
    cX_te = np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32")
    X.append(cX)
    X_te.append(cX_te)
X = np.hstack(X)
X_te = np.hstack(X_te)
    
Y = np.load(f'{featdir}/{subject}_each_{target}_tr.npy').astype("float32").reshape([X.shape[0],-1])
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32").reshape([X_te.shape[0],-1])
    
print(f'Now making decoding model for... {subject}:  {roi}, {target}')
print(f'X {X.shape}, Y {Y.shape}, X_te {X_te.shape}, Y_te {Y_te.shape}')
pipeline.fit(X, Y)
scores = pipeline.predict(X_te)
rs = correlation_score(Y_te.T,scores.T)
print(f'Prediction accuracy is: {np.mean(rs):3.3}')

np.save(f'{savedir}/{subject}_{"_".join(roi)}_scores_{target}.npy',scores)

Now making decoding model for... subj01:  early, init_latent
X (24980, 5917), Y (24980, 6400), X_te (982, 5917), Y_te (982, 6400)
Prediction accuracy is: 0.239


In [10]:
target  = 'c'
subject = 'subj01'
featdir = '/hdd/yuchen/nsdfeat/subjfeat'
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32")
scores = np.load('/hdd/yuchen/decoded/subj01/subj01_v_e_n_t_r_a_l_scores_c.npy')

from sklearn.metrics import mean_squared_error
mean_squared_error(Y_te,scores)
# mse for text

0.6907157

In [30]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_te,scores)
# mse for image

0.61325544

## diffusion_decoding.py 

In [5]:
import h5py
from PIL import Image
import scipy.io
import argparse, os
import pandas as pd
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from tqdm import trange
from einops import rearrange
from torch import autocast
from contextlib import nullcontext
from pytorch_lightning import seed_everything
import sys
# sys.path.append("../utils/")
# from nsd_access.nsda import NSDAccess
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler

In [6]:
from os.path import join as pjoin

betas_csv_dir = '/hdd/yuchen/cs292/fmri/betas_csv/'
sub = '01'
    
data_file = pjoin(betas_csv_dir, f'sub-{sub}_ResponseData.h5')
responses = pd.read_hdf(data_file)
vox_f = pjoin(betas_csv_dir, f'sub-{sub}_VoxelMetadata.csv')
voxdata = pd.read_csv(vox_f)
stim_f = pjoin(betas_csv_dir, f'sub-{sub}_StimulusMetadata.csv')
stimdata = pd.read_csv(stim_f)

test_idx = stimdata[stimdata['trial_type'] == 'test'].index.tolist()
    
test_fmri = responses[test_idx]
    
test_labels = stimdata[stimdata['trial_type'] == 'test']['stimulus']
    
roi_idx = voxdata[(voxdata['V1'] == 1) ]['voxel_id'].tolist()
test_fmri_temp = test_fmri.iloc[roi_idx]

roi_neuron = {'V1': test_fmri_temp.shape[0]}

for roi in ['V2', 'V3', 'hV4']:
    roi_idx = voxdata[(voxdata[roi] == 1)]['voxel_id'].tolist()
    test_fmri_temp = pd.concat([test_fmri_temp, test_fmri.iloc[roi_idx]])
    roi_neuron[roi] = test_fmri.iloc[roi_idx].shape[0]
    
test_fmri = test_fmri_temp

test = stimdata[stimdata['trial_type'] == 'test']
test_fmri_avg = []
test_labels = test['stimulus'].unique()
for stim in test_labels:
    repeated_trial = test[test['stimulus'] == stim].index.tolist()
    test_fmri_avg.append(test_fmri[repeated_trial].mean(axis=1).to_numpy())
test_fmri_avg = np.array(test_fmri_avg)

del responses, voxdata, stimdata

betas_te_avg = test_fmri_avg

In [7]:
len(test_labels)

100

In [8]:
def load_model_from_config(config, ckpt, gpu, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda(f"cuda:{gpu}")
    model.eval()
    return model

def load_img_from_arr(img_arr):
    image = Image.fromarray(img_arr).convert("RGB")
    w, h = 512, 512
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [9]:
class Opt:
    subject = 'subj01'
    root_folder = '/hdd/yuchen/'
    imgidx = (0, 10)
    gpu = 1
    seed = 42
    featname = ''  # init_latent or c
    use_stim = ''  # each or ave

    target = ''  # init_latent or c
    roi = ''  # ventral  or early 
    
    method = 'cvpr'
    
opt = Opt()

In [10]:
imgidx = opt.imgidx
gpu = opt.gpu
method = opt.method
subject=opt.subject
torch.cuda.set_device(opt.gpu)

# captdir = f'{opt.root_folder}decoded/{subject}/captions/'

#     # Load NSD information
# nsd_expdesign = scipy.io.loadmat(f'{opt.root_folder}nsd/nsddata/experiments/nsd/nsd_expdesign.mat')

#     # Note that mos of them are 1-base index!
#     # This is why I subtract 1
# sharedix = nsd_expdesign['sharedix'] -1 

# nsda = NSDAccess(f'{opt.root_folder}nsd/')
# sf = h5py.File(nsda.stimuli_file, 'r')
# sdataset = sf.get('imgBrick')

# stims_ave = np.load(f'{opt.root_folder}mrifeat/{subject}/{subject}_stims_ave.npy')

In [11]:
# tr_idx = np.zeros_like(stims_ave)
# for idx, s in enumerate(stims_ave):
#     if s in sharedix:
#         tr_idx[idx] = 0
#     else:
#         tr_idx[idx] = 1

In [12]:
    # Load Stable Diffusion Model
path = '/hdd/yuchen/StableDiffusionReconstruction/codes/diffusion_sd1'
config = path + '/stable-diffusion/configs/stable-diffusion/v1-inference.yaml'
ckpt = path + '/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt'
config = OmegaConf.load(f"{config}")
torch.cuda.set_device(gpu)
model = load_model_from_config(config, f"{ckpt}", gpu)

n_samples = 1
ddim_steps = 50
ddim_eta = 0.0
strength = 0.8
scale = 5.0
n_iter = 5
precision = 'autocast'
precision_scope = autocast if precision == "autocast" else nullcontext
batch_size = n_samples
device = torch.device(f"cuda:{opt.gpu}") if torch.cuda.is_available() else torch.device("cpu")

Loading model from /hdd/yuchen/StableDiffusionReconstruction/codes/diffusion_sd1/stable-diffusion/models/ldm/stable-diffusion-v1/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode


/hdd/yuchen/anaconda3/envs/ldm/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.9.self_attn.v_proj.weight', 'vision_model.encoder.layers.2.layer_norm2.weight', 'vision_model.pre_layrnorm.weight', 'vision_model.encoder.layers.7.mlp.fc2.weight', 'vision_model.encoder.layers.17.mlp.fc2.weight', 'vision_model.encoder.layers.20.mlp.fc2.weight', 'vision_model.encoder.layers.5.mlp.fc1.weight', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.4.mlp.fc1.bias', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.9.self_attn.q_proj.bias', 'vision_model.encoder.layers.11.self_attn.q_proj.bias', 'vision_model.encoder.layers.15.layer_norm2.bias', 'vision_model.encoder.layers.8.mlp.fc1.bias', 'vision_model.encoder.layers.17.layer_norm1.weight', 'visual_projection.weight', 'vision_model.encoder.layers.20.self_attn.q_proj.weight', 'vision_model.encoder.layers.3.self_attn.v

In [13]:
outdir = f'{opt.root_folder}decoded/image-{method}/{subject}/'
os.makedirs(outdir, exist_ok=True)

sample_path = os.path.join(outdir, f"samples")
os.makedirs(sample_path, exist_ok=True)
precision = 'autocast'
device = torch.device(f"cuda:{gpu}") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)
sampler = DDIMSampler(model)

sampler.make_schedule(ddim_num_steps=ddim_steps, ddim_eta=ddim_eta, verbose=False)

assert 0. <= strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(strength * ddim_steps)
print(f"target t_enc is {t_enc} steps")


target t_enc is 40 steps


In [18]:
scores_c.shape

(100, 11520)

In [19]:
%%capture
scores_c = np.load('/hdd/yuchen/things_baseline_c.npy').reshape(100, 15, 768)
scores_latent = np.load('/hdd/yuchen/things_baseline_latent_init.npy')
sample_path = '/hdd/yuchen/decoded/image-cvpr/subj01/things-baseline/samples/'

num_test_img = len(test_labels)

for imgidx in range(0, num_test_img):
# Load z (Image)
    # imgidx_te = np.where(tr_idx==0)[0][imgidx] # Extract test image index
    # idx73k= stims_ave[imgidx_te
    img_path = '/hdd/yuchen/cs292/images/' + test_labels[imgidx]
    with Image.open(img_path) as img:
        img.save(os.path.join(sample_path, f"{imgidx:05}_org.png"))
    # Image.fromarray(np.squeeze(sdataset[idx73k,:,:,:]).astype(np.uint8)).save(
    #     os.path.join(sample_path, f"{imgidx:05}_org.png")) 
        
    if method in ['cvpr','text']:
        # roi_latent = 'early'
        roi_latent = 'e_a_r_l_y'
        
        # scores_latent = np.load(f'{opt.root_folder}decoded/{subject}/{subject}_{roi_latent}_scores_init_latent.npy')
        imgarr = torch.Tensor(scores_latent[imgidx,:].reshape(4,40,40)).unsqueeze(0).to(f'cuda:{opt.gpu}')
    
        # Generate image from Z
        precision_scope = autocast if precision == "autocast" else nullcontext
        with torch.no_grad():
            with precision_scope("cuda"):
                with model.ema_scope():
                    x_samples = model.decode_first_stage(imgarr)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
        im = Image.fromarray(x_sample.astype(np.uint8)).resize((512,512))
        
        im = np.array(im)
    
    init_image = load_img_from_arr(im).to(f'cuda:{opt.gpu}')
    init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
    
    # Load c (Semantics)
    if method == 'cvpr':
        # roi_c = 'ventral'
        roi_c = 'v_e_n_t_r_a_l'
        # scores_c = np.load(f'{opt.root_folder}decoded/{subject}/{subject}_{roi_c}_scores_c.npy')
        # carr = scores_c[imgidx,:].reshape(77,768)
        carr = scores_c[imgidx,:]
        c = torch.Tensor(carr).unsqueeze(0).to(f'cuda:{opt.gpu}')
    
        # Generate image from Z (image) + C (semantics)
    base_count = 0
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for n in trange(n_iter, desc="Sampling"):
                    uc = model.get_learned_conditioning(batch_size * [""])
    
                        # encode (scaled latent)
                    z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        # decode it
                    # samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                    samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=scale,
                                                unconditional_conditioning=uc,)
    
                    x_samples = model.decode_first_stage(samples)
                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)
    
                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                    Image.fromarray(x_sample.astype(np.uint8)).save(
                        os.path.join(sample_path, f"{imgidx:05}_{base_count:03}.png"))    
                    base_count += 1

    # if imgidx == 20:
    #     break


In [63]:
z_enc[].shape

torch.Size([1, 4, 64, 64])

In [46]:
imgarr.shape

torch.Size([1, 4, 16, 16])